# Data Acquisition

### This consist of 3 parts:
1. The FourSquare Top 30 Venues to Visit in Chicago
2. For each of the Top Site get a list of up restaurants in the surrounding area
3. The Chicago Police Department Crime Data for the last Year

### Import Libraries

Before we get the data and start exploring it, let's download all the dependencies and libraries that we will need.

In [1]:
import pandas as pd # Import pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests # Import Requests

from bs4 import BeautifulSoup # Import BeautifulSoup

In [2]:
import yaml # Yaml is used to store some of the required configurations

with open("./Data/config.yaml", "r") as f:
    cfg = yaml.load(f)
 
search_params = {
    'client_id': cfg['client_id'],
    'client_secret': cfg['client_secret'],
    'intent': 'browse',
    'limit': 50,
    'v': cfg['version']
}

<ipython-input-2-3f679cce538e>:4: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  cfg = yaml.load(f)


## FourSquare Top 30 Venues to Visit in Chicago
To get the top venues in a city we can use the FourSquare website directly to request the top sites in Chicago and then use BeautifulSoup to scrape the data we need. Once we have this starting data the other supplemental data we need to complete this dataset can be retrieved from using the FourSquare Venue API.

In [3]:
# Use the Requests get method to request the top sites in Chicago
page = requests.get( 
    "https://foursquare.com/explore?mode=url&near=Chicago%2C%20IL%2C%20United%20States&nearGeoId=72057594042815334&q=Top%20Picks")

soup = BeautifulSoup(page.content, 'html.parser') # Convert the HTML response into a BeautifulSoup Object

top_venues = soup.find_all('div', class_='venueDetails') # Use the BeautifulSoup find_all method to extract each top site venue details.

From this HTML the following data will be extracted:

* Venue Name
* Venue Score
* Venue Category
* Venue HREF
* Venue ID (Extracted from the HREF)

### Creating Dataframe for Top Venues

The `top_venues` list, a sample of which is shown above, only contains some of the data required. In addition to the attributes extracted directly from the HTML code the following attributes are also required:
* Venue Address
* Venue Postalcode
* Venue City
* Venue Latitude
* Venue Longitude

These attributes will be obtained directly from FourSquare using the `venues` API. The process is as follows:
1. Create a new empty Pandas dataframe to hold the data for the Top Sites / Venues
1. Extract the available attributes from the HTML code
1. For each venue
    1. Contruct a URL to interagate the FourSquare Venue API for each top site
    1. Using the `venues` API and the URL request the data from FourSquare
    1. Get the properly formatted address and the latitude and longitude data from the returned JSON
    1. Write the data for each venue to the top venues dataframs

In [4]:
# The column names for the top venues dataframe
venue_columns = ['id', 
                 'score', 
                 'category', 
                 'name', 
                 'address',
                 'postalcode',
                 'city',
                 'href', 
                 'latitude', 
                 'longitude']

# Create the empty top venues dataframe
df_top_venues = pd.DataFrame(columns=venue_columns)

# For each venue in the BeautifulSoup HTML object
for venue in top_venues:
    
    # Extract the available attributes
    venue_name = venue.find(target="_blank").get_text()
    venue_score = venue.find(class_="venueScore positive").get_text()
    venue_cat = venue.find(class_="categoryName").get_text()
    venue_href = venue.find(class_="venueName").h2.a['href']
    venue_id = venue_href.split('/')[-1]

    if 'promotedTipId' in venue_id: 
        continue
        
    # Contruct the FourSquare venue API URL
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(
        venue_id, 
        cfg['client_id'],
        cfg['client_secret'],
        cfg['version'])
    
    # Request the venue data
    result = requests.get(url).json()
        
    # Get the properly formatted address and the latitude and longitude
    venue_address = result['response']['venue']['location']['address']
    venue_postalcode = result['response']['venue']['location']['postalCode']
    venue_city = result['response']['venue']['location']['city']
    venue_latitude = result['response']['venue']['location']['lat']
    venue_longitude = result['response']['venue']['location']['lng']
    
    # Add the venue to the top venues dataframe
    df_top_venues = df_top_venues.append({'id': venue_id,
                                          'score': venue_score,
                                          'category': venue_cat,
                                          'name': venue_name,
                                          'address': venue_address,
                                          'postalcode': venue_postalcode,
                                          'city': venue_city,
                                          'href': venue_href,
                                          'latitude': venue_latitude,
                                          'longitude': venue_longitude}, ignore_index=True)

In [5]:
df_top_venues.shape # Verify the shape of the top venues dataframe

(30, 10)

In [6]:
df_top_venues.dtypes # Verify the dtypes of the top venues dataframe

id             object
score          object
category       object
name           object
address        object
postalcode     object
city           object
href           object
latitude      float64
longitude     float64
dtype: object

In [7]:
df_top_venues['score'] = pd.to_numeric(df_top_venues['score'], errors='coerce').fillna(0) # The score type needs to be converted to float

df_top_venues.score.describe() # Describe the score to see if there is nuch variance in the values

count    30.000000
mean      9.420000
std       0.080516
min       9.300000
25%       9.400000
50%       9.400000
75%       9.500000
max       9.600000
Name: score, dtype: float64

In [8]:
df_top_venues.head() # Review the head of the dataframe to make sure it looks as expected

,id,score,category,name,address,postalcode,city,href,latitude,longitude
0,42b75880f964a52090251fe3,9.6,Park,Millennium Park,201 E Randolph St,60601,Chicago,/v/millennium-park/42b75880f964a52090251fe3,41.883112,-87.623851
1,49e9ef74f964a52011661fe3,9.5,Art Museum,The Art Institute of Chicago,111 S Michigan Ave,60603,Chicago,/v/the-art-institute-of-chicago/49e9ef74f964a5...,41.879610,-87.623552
2,4c47533649fa9521cb1f5e62,9.5,Park,Grant Park,337 E Randolph Dr,60601,Chicago,/v/grant-park/4c47533649fa9521cb1f5e62,41.876626,-87.619263
3,4b9d15c5f964a520478e36e3,9.5,Waterfront,Chicago Riverwalk,Chicago River,60601,Chicago,/v/chicago-riverwalk/4b9d15c5f964a520478e36e3,41.887280,-87.627217
4,4b06c066f964a52097ef22e3,9.5,Liquor Store,Binny's Beverage Depot,1720 N Marcey St,60614,Chicago,/v/binnys-beverage-depot/4b06c066f964a52097ef22e3,41.913048,-87.655320


In [9]:
df_top_venues.to_pickle('./Pickles/top_venues.pkl') # Finally write the dataframe to a pickle file for restoring later

## FourSquare Restaurent Recommendations Data

Using the the list of all venue id values in the Top Sites DataFrame and the FourSquare categoryID that represents all food venues, we now search for restaurants within a 500 meter radius.

The requests returns a JSON object which can then be queried for the restaurant details required.From this JSON the following attributes are extraced and added to the Dataframe:

* Restaurant ID
* Restaurant Category Name
* Restaurant Category ID
* Restaurant Nest_name
* Restaurant Address
* Restaurant Postalcode
* Restaurant City
* Restaurant Latitude
* Restaurant Longitude
* Venue Name
* Venue Latitude
* Venue Longitude

The only piece of data that is missing is the Score or Rating of the Restaurant. To get this we need to make another FourSquare API query using the id of the Restaurant.

Using just the data in this DataFrame we will be able to generate maps displaying the chosen Top List Venue and the best scored surrounding restaurants.

In [10]:
#The column names for the restaurants dataframe
restaurants_columns = ['id',
                       'score', 
                       'category', 
                       'categoryID', 
                       'name', 
                       'address',
                       'postalcode',
                       'city',
                       'latitude',
                       'longitude', 
                       'venue_name', 
                       'venue_latitude',
                       'venue_longitude']

# Create the empty top venues dataframe
df_restaurant = pd.DataFrame(columns=restaurants_columns)

# Create a list of all the top venue latitude and longitude
top_venue_lats = df_top_venues['latitude'].values
top_venue_lngs = df_top_venues['longitude'].values

# Create a list of all the top venue names
top_venue_names = df_top_venues['name'].values

# Iterate over each of the top venues
# The venue name, latitude and longitude are passed to the loop
for ven_name, ven_lat, ven_long in zip(top_venue_names, top_venue_lats, top_venue_lngs):
    
    # Configure additional Search parameters
    # This is the FourSquare Category Id for all food venues
    categoryId = '4d4b7105d754a06374d81259'
    radius = 100
    limit = 20
    
    # Contruct the FourSquare search API URL
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}&limit={}'.format(
        cfg['client_id'],
        cfg['client_secret'],
        ven_lat,
        ven_long,
        cfg['version'],
        categoryId,
        radius,
        limit)
    
    # Make the search request
    results = requests.get(url).json()
    
    # Want a good selection of Restaurents
    # If less than 10 are returned ignore
    if len(results['response']['venues']) < 10:
        continue
        
    # Populate the new dataframe with the list of restaurants
    # Get the values for each Restaurant from the JSON
    for restaurant in results['response']['venues']:
 
        # Sometimes the Venue JSON is missing data. If so ignore and continue
        try:
            # Get location details
            rest_id = restaurant['id']
            rest_category = restaurant['categories'][0]['pluralName']
            rest_categoryID = restaurant['categories'][0]['id']
            rest_name = restaurant['name']
            rest_address = restaurant['location']['address']
            rest_postalcode = restaurant['location']['postalCode']
            rest_city = restaurant['location']['city']
            rest_latitude = restaurant['location']['lat']
            rest_longitude = restaurant['location']['lng']
            
            # Contruct the FourSquare venue API URL to get the venues rating / score
            rest_url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(
                rest_id, 
                cfg['client_id'],
                cfg['client_secret'],
                cfg['version'])

            # Get the restaurant score and href
            result = requests.get(rest_url).json()
            rest_score = result['response']['venue']['rating']
            
            # Add the restaurant details to the dataframe
            df_restaurant = df_restaurant.append({'id': rest_id,
                                                  'score': rest_score,
                                                  'category': rest_category,
                                                  'categoryID': rest_categoryID,
                                                  'name': rest_name,
                                                  'address': rest_address,
                                                  'postalcode': rest_postalcode,
                                                  'city': rest_city,
                                                  'latitude': rest_latitude,
                                                  'longitude': rest_longitude,
                                                  'venue_name': ven_name,
                                                  'venue_latitude': ven_lat,
                                                  'venue_longitude': ven_long}, ignore_index=True)
            
        # If there are any issue with a restaurant ignore and continue
        except:
            continue
            

In [11]:
df_restaurant.shape # Verify the shape of the restaurants dataframe

(91, 13)

In [12]:
df_restaurant.dtypes # Verify the dtypes of the restaurants dataframe

id                  object
score              float64
category            object
categoryID          object
name                object
address             object
postalcode          object
city                object
latitude           float64
longitude          float64
venue_name          object
venue_latitude     float64
venue_longitude    float64
dtype: object

In [13]:
df_restaurant.head() # Review the head of the dataframe to make sure it looks as expected

,id,score,category,categoryID,name,address,postalcode,city,latitude,longitude,venue_name,venue_latitude,venue_longitude
0,4582cc9ef964a520843f1fe3,6.2,American Restaurants,4bf58dd8d48988d14e941735,Park Grill,11 N Michigan Ave,60602,Chicago,41.882732,-87.623847,Millennium Park,41.883112,-87.623851
1,551192b1498e8695352d33e6,8.1,American Restaurants,4bf58dd8d48988d14e941735,Remington's,20 N Michigan Ave,60602,Chicago,41.882628,-87.624608,Millennium Park,41.883112,-87.623851
2,4bf98a56508c0f4793663f31,6.7,American Restaurants,4bf58dd8d48988d14e941735,The Plaza at Park Grill,11 N Michigan Ave,60602,Chicago,41.882668,-87.623769,Millennium Park,41.883112,-87.623851
3,54060b96498ed2702c0632f7,8.3,Coffee Shops,4bf58dd8d48988d1e0931735,Peet's Coffee & Tea,"20 North Michigan Ave.,",60602,Chicago,41.882467,-87.624686,Millennium Park,41.883112,-87.623851
4,4e73881c922e0a374024c7b8,7.7,Coffee Shops,4bf58dd8d48988d1e0931735,Starbucks,8 N Michigan Ave,60602,Chicago,41.882478,-87.624701,Millennium Park,41.883112,-87.623851


In [14]:
df_restaurant.score.describe() # Describe the score to see if there is nuch variance in the values

count    91.000000
mean      7.679121
std       0.989895
min       5.500000
25%       6.750000
50%       7.700000
75%       8.400000
max       9.500000
Name: score, dtype: float64

In [15]:
df_restaurant.venue_name.nunique() # How many of the top 30 sites / venues had > 10 restaurants nearby

9

In [16]:
df_restaurant.category.nunique() # How many unique restaurant categories are there

38

In [17]:
df_restaurant.name.nunique() # How many unique restaurants are there

79

In [18]:
df_restaurant.groupby('category')['name'].count().sort_values(ascending=False)[:10] # What arethe top 10 most frequently occuring restaurant types

category
American Restaurants        9
Sandwich Places             8
Chinese Restaurants         7
Coffee Shops                7
Cafés                       6
Seafood Restaurants         4
Bakeries                    4
Italian Restaurants         4
New American Restaurants    3
Burger Joints               3
Name: name, dtype: int64

In [19]:
df_restaurant.groupby('category')['score'].mean().sort_values(ascending=False)[:10] # Which restaurants have to highest average score

category
Delis / Bodegas                     9.50
Mediterranean Restaurants           9.20
Bars                                9.00
Latin American Restaurants          9.00
Gourmet Shops                       8.85
New American Restaurants            8.80
Ramen Restaurants                   8.70
Southern / Soul Food Restaurants    8.60
Breweries                           8.50
Seafood Restaurants                 8.40
Name: score, dtype: float64

In [20]:
df_restaurant.to_pickle('./Pickles/restaurants.pkl') # Finally write the dataframe to a pickle file for restoring later

## Chicago Crime DataSet

This dataset reflects reported incidents of crime (with the exception of murders where data exists for each victim) that occurred in the City of Chicago in the last year, minus the most recent seven days. Data is extracted from the Chicago Police Department's CLEAR (Citizen Law Enforcement Analysis and Reporting) system. In order to protect the privacy of crime victims, addresses are shown at the block level only and specific locations are not identified.

| Column Name   | Type          | Description                                            | 
| :------------ | :------------ | :----------------------------------------------------- | 
| Case Number    | Plain Text    | The Chicago Police Department RD Number (Records Division Number), which is unique to the incident. | 
| Date | Date & Time   | Date when the incident occurred. this is sometimes a best estimate. |
| Block	        | Plain Text    | The partially redacted address where the incident occurred, placing it on the same block as the actual address. |
| IUCR	        | Plain Text    | The Illinois Unifrom Crime Reporting code. This is directly linked to the Primary Type and Description. See the list of IUCR codes at https://data.cityofchicago.org/d/c7ck-438e. |
| Primary Type   | Plain Text    | The primary description of the IUCR code. |
| Description	| Plain Text    | The secondary description of the IUCR code, a subcategory of the primary description. |
| Location Description | Plain Text | Description of the location where the incident occurred. |
| Arrest        | Plain Text    | Indicates whether an arrest was made. |
| Domestic      | Plain Text    | Indicates whether the incident was domestic-related as defined by the Illinois Domestic Violence Act. |
| Beat          | Plain Text    | Indicates the beat where the incident occurred. A beat is the smallest police geographic area – each beat has a dedicated police beat car. Three to five beats make up a police sector, and three sectors make up a police district. The Chicago Police Department has 22 police districts. See the beats at https://data.cityofchicago.org/d/aerh-rz74. |
| Ward	        | Number        | The ward (City Council district) where the incident occurred. See the wards at https://data.cityofchicago.org/d/sp34-6z76. |
| FBI Code        | Plain Text    | Indicates the crime classification as outlined in the FBI's National Incident-Based Reporting System (NIBRS). See the Chicago Police Department listing of these classifications at http://gis.chicagopolice.org/clearmap_crime_sums/crime_types.html. |
| X Coordinate	| Plain Text    | The x coordinate of the location where the incident occurred in State Plane Illinois East NAD 1983 projection. This location is shifted from the actual location for partial redaction but falls on the same block. |
| Y Coordinate	| Plain Text    | The y coordinate of the location where the incident occurred in State Plane Illinois East NAD 1983 projection. This location is shifted from the actual location for partial redaction but falls on the same block. |
| Latitude	    | Number        | The latitude of the location where the incident occurred. This location is shifted from the actual location for partial redaction but falls on the same block. |
| Longitude	    | Number        | The longitude of the location where the incident occurred. This location is shifted from the actual location for partial redaction but falls on the same block. |
| Location	    | Location      | The location where the incident occurred in a format that allows for creation of maps and other geographic operations on this data portal. This location is shifted from the actual location for partial redaction but falls on the same block. |
	


### Import the 2019 DataSet

The full dataset, 2001 to 2019 contains over 7.03M rows. This makes processing the dataset difficult and time consuming. Only the 2019 data, 242720 records, will be used. In addition, the following clean-up steps are required:

Not all of the columns are required. The following columns are removed:
* IUCR
* ARREST
* DOMESTIC
* BEAT
* WARD
* FBI CD
* X COORDINATE
* Y COORDINATE
* LOCATION

In [33]:
# These are the columns that we want to keep. Others will not be imported.
crime_keep_columns = ['Case Number',
                      'Date',
                      'Block', 
                      'Primary Type',
                      'Ward',
                      'Latitude',
                      'Longitude']

In [29]:
# Download csv
!wget -O './Data/crimes.csv' https://data.cityofchicago.org/api/views/w98m-zvie/rows.csv?accessType=DOWNLOAD

--2019-12-18 15:47:27--  https://data.cityofchicago.org/api/views/w98m-zvie/rows.csv?accessType=DOWNLOAD
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving data.cityofchicago.org (data.cityofchicago.org)... 52.206.68.26, 52.206.140.205, 52.206.140.199
Connecting to data.cityofchicago.org (data.cityofchicago.org)|52.206.68.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘./Data/crimes.csv’

./Data/crimes.csv       [            <=>     ]  55.16M   552KB/s    in 1m 46s  

2019-12-18 15:49:15 (533 KB/s) - ‘./Data/crimes.csv’ saved [57843705]



In [34]:
df = pd.read_csv('./Data/crimes.csv',
                 usecols=crime_keep_columns) # Read cvs download into datafram